# 模組 3: 企業級品質控制與監控系統
## Module 3: Enterprise Quality Control & Monitoring Systems

> **教學目標**: 建立多維度品質評估框架、實作自動化品質監控、設計異常檢測與警報系統

---

## 📚 學習成果 (Learning Outcomes)

完成本模組後，您將能夠：
1. **設計多維度品質模型** - 基於 ISO 25012 標準的全面評估框架
2. **實作自動化品質監控** - 即時檢測品質變化與異常
3. **建立品質治理流程** - 包含審核、修正、版本控制
4. **設計異常檢測系統** - 機器學習驅動的品質異常識別

---

## 🎯 核心概念 (Core Concepts)

### 1. 資料品質維度 (ISO 25012)
- **準確性 (Accuracy)**: 資料與真實值的符合程度
- **完整性 (Completeness)**: 必要資訊的完整程度
- **一致性 (Consistency)**: 資料格式與規則的一致性
- **時效性 (Currency)**: 資料的新鮮度與及時性
- **可理解性 (Understandability)**: 資料的可讀性與清晰度
- **可追溯性 (Traceability)**: 資料來源與處理歷程的追蹤

### 2. 品質控制策略
- **預防性控制**: 在資料攝取階段的品質檢查
- **檢測性控制**: 處理過程中的品質監控
- **修正性控制**: 發現問題後的自動修復
- **持續改進**: 基於歷史資料的品質優化

### 3. 異常檢測技術
- **統計異常檢測**: 基於分佈和閾值的檢測
- **機器學習異常檢測**: 使用 Isolation Forest、One-Class SVM
- **時間序列異常檢測**: 檢測品質指標的時間變化
- **多變量異常檢測**: 考慮多個品質維度的關聯

## 🔧 環境設定與依賴安裝

In [ ]:
# 安裝核心依賴套件
!pip install scikit-learn pandas numpy matplotlib seaborn plotly
!pip install textstat language-tool-python pyspellchecker
!pip install sqlalchemy pydantic fastapi
!pip install schedule APScheduler
!pip install anomaly-detection-toolkit pyod

# 機器學習與統計
!pip install scipy statsmodels

# 時間序列分析
!pip install prophet fbprophet --quiet || echo "Prophet installation failed, continuing..."

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [ ]:
# 導入必要的函式庫
import os
import sys
import json
import sqlite3
import logging
import warnings
from pathlib import Path
from datetime import datetime, timedelta, timezone
from typing import List, Dict, Optional, Tuple, Any, Union
from dataclasses import dataclass, field, asdict
from enum import Enum
from collections import defaultdict, deque
import warnings
warnings.filterwarnings('ignore')

# 資料處理與分析
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.api as sm
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

# 文本品質分析
import textstat
import language_tool_python
from spellchecker import SpellChecker
import re

# 資料庫與資料模型
from sqlalchemy import create_engine, Column, String, Integer, Float, DateTime, Text, JSON, Boolean
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from pydantic import BaseModel, Field, validator

# 可視化
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 任務調度
import schedule
import time
import threading

# 設定專案路徑
PROJECT_ROOT = Path('/home/os-sunnie.gd.weng/python_workstation/side-project/RAG/data_governance/kms_governance')
sys.path.append(str(PROJECT_ROOT))

# 配置日誌
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(PROJECT_ROOT / 'quality_control.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger('QualityControl')

print("✅ 環境設定完成")
print(f"📂 專案根目錄: {PROJECT_ROOT}")

✅ 環境設定完成
📂 專案根目錄: /home/os-sunnie.gd.weng/python_workstation/side-project/RAG/data_governance/kms_governance


## 📋 品質控制資料模型

### 品質指標定義

In [ ]:
# SQLAlchemy 基礎設定
Base = declarative_base()

class QualityDimension(Enum):
    """品質維度枚舉"""
    ACCURACY = "accuracy"
    COMPLETENESS = "completeness"
    CONSISTENCY = "consistency"
    CURRENCY = "currency"
    UNDERSTANDABILITY = "understandability"
    TRACEABILITY = "traceability"

class QualityStatus(Enum):
    """品質狀態枚舉"""
    EXCELLENT = "excellent"  # > 0.9
    GOOD = "good"           # 0.7 - 0.9
    FAIR = "fair"           # 0.5 - 0.7
    POOR = "poor"           # < 0.5

class AlertSeverity(Enum):
    """警報嚴重程度"""
    LOW = "low"
    MEDIUM = "medium"
    HIGH = "high"
    CRITICAL = "critical"

@dataclass
class QualityMetrics:
    """品質指標資料類別"""
    accuracy: float = 0.0
    completeness: float = 0.0
    consistency: float = 0.0
    currency: float = 0.0
    understandability: float = 0.0
    traceability: float = 0.0
    overall_score: float = 0.0
    
    def to_dict(self) -> Dict[str, float]:
        return asdict(self)
    
    def get_status(self) -> QualityStatus:
        """根據綜合分數判斷品質狀態"""
        if self.overall_score > 0.9:
            return QualityStatus.EXCELLENT
        elif self.overall_score > 0.7:
            return QualityStatus.GOOD
        elif self.overall_score > 0.5:
            return QualityStatus.FAIR
        else:
            return QualityStatus.POOR

@dataclass
class QualityRule:
    """品質規則定義"""
    rule_id: str
    name: str
    description: str
    dimension: QualityDimension
    threshold: float
    weight: float = 1.0
    active: bool = True

# 品質評估記錄表
class QualityAssessment(Base):
    """品質評估記錄表"""
    __tablename__ = 'quality_assessments'
    
    assessment_id = Column(String(50), primary_key=True)
    document_id = Column(String(50), nullable=False, index=True)
    chunk_id = Column(String(50), index=True)  # 可選，針對分塊的評估
    
    # 品質指標分數
    accuracy_score = Column(Float, nullable=False)
    completeness_score = Column(Float, nullable=False)
    consistency_score = Column(Float, nullable=False)
    currency_score = Column(Float, nullable=False)
    understandability_score = Column(Float, nullable=False)
    traceability_score = Column(Float, nullable=False)
    overall_score = Column(Float, nullable=False, index=True)
    
    # 詳細資訊
    quality_status = Column(String(20), nullable=False, index=True)
    assessment_details = Column(JSON)  # 詳細的評估結果
    
    # 評估配置
    assessor_version = Column(String(20), default='1.0')
    rules_applied = Column(JSON)  # 應用的品質規則
    
    # 時間資訊
    assessed_at = Column(DateTime, default=datetime.now, index=True)
    processing_time_ms = Column(Integer)
    
    # 備註
    notes = Column(Text)

# 品質異常記錄表
class QualityAnomaly(Base):
    """品質異常記錄表"""
    __tablename__ = 'quality_anomalies'
    
    anomaly_id = Column(String(50), primary_key=True)
    document_id = Column(String(50), nullable=False, index=True)
    assessment_id = Column(String(50), nullable=False)
    
    # 異常資訊
    anomaly_type = Column(String(50), nullable=False)  # score_drop, threshold_breach, etc.
    affected_dimension = Column(String(50), nullable=False)
    anomaly_score = Column(Float, nullable=False)  # 異常程度分數
    
    # 閾值資訊
    expected_value = Column(Float)
    actual_value = Column(Float)
    threshold_value = Column(Float)
    
    # 檢測方法
    detection_method = Column(String(50), nullable=False)
    detector_config = Column(JSON)
    
    # 狀態管理
    status = Column(String(20), default='open')  # open, investigating, resolved, false_positive
    severity = Column(String(20), nullable=False)
    
    # 時間資訊
    detected_at = Column(DateTime, default=datetime.now, index=True)
    resolved_at = Column(DateTime)
    
    # 處理資訊
    assigned_to = Column(String(100))
    resolution_notes = Column(Text)

# 品質趨勢表
class QualityTrend(Base):
    """品質趨勢分析表"""
    __tablename__ = 'quality_trends'
    
    trend_id = Column(String(50), primary_key=True)
    
    # 時間窗口
    period_start = Column(DateTime, nullable=False, index=True)
    period_end = Column(DateTime, nullable=False)
    period_type = Column(String(20), nullable=False)  # daily, weekly, monthly
    
    # 統計資訊
    document_count = Column(Integer, nullable=False)
    avg_overall_score = Column(Float, nullable=False)
    min_score = Column(Float, nullable=False)
    max_score = Column(Float, nullable=False)
    std_score = Column(Float, nullable=False)
    
    # 分維度統計
    dimension_averages = Column(JSON, nullable=False)
    
    # 分佈統計
    status_distribution = Column(JSON)  # excellent, good, fair, poor 的分佈
    type_distribution = Column(JSON)    # 不同文檔類型的品質分佈
    
    # 異常統計
    anomaly_count = Column(Integer, default=0)
    critical_anomaly_count = Column(Integer, default=0)
    
    # 計算時間
    calculated_at = Column(DateTime, default=datetime.now)

print("✅ 品質控制資料模型定義完成")

✅ 品質控制資料模型定義完成


### Pydantic 驗證模型

In [ ]:
# Pydantic 模型用於 API 和資料驗證

class QualityAssessmentRequest(BaseModel):
    """品質評估請求模型"""
    document_id: str = Field(..., min_length=1)
    chunk_id: Optional[str] = None
    content: str = Field(..., min_length=1)
    force_reassessment: bool = Field(default=False)
    custom_rules: Optional[List[Dict]] = None
    
    @validator('content')
    def validate_content_length(cls, v):
        if len(v.strip()) < 10:
            raise ValueError('Content too short for quality assessment')
        return v

class QualityThreshold(BaseModel):
    """品質閾值設定模型"""
    dimension: str = Field(..., pattern="^(accuracy|completeness|consistency|currency|understandability|traceability|overall)$")
    min_acceptable: float = Field(..., ge=0.0, le=1.0)
    target_value: float = Field(..., ge=0.0, le=1.0)
    weight: float = Field(default=1.0, ge=0.0, le=10.0)
    
    @validator('target_value')
    def target_must_be_higher_than_min(cls, v, values):
        if 'min_acceptable' in values and v < values['min_acceptable']:
            raise ValueError('Target value must be >= min_acceptable')
        return v

class AnomalyDetectionConfig(BaseModel):
    """異常檢測配置模型"""
    method: str = Field(..., pattern="^(statistical|isolation_forest|one_class_svm|zscore)$")
    sensitivity: float = Field(default=0.1, ge=0.01, le=0.5)
    min_samples: int = Field(default=10, ge=5)
    lookback_days: int = Field(default=30, ge=1, le=365)
    enable_multivariate: bool = Field(default=True)

class QualityReport(BaseModel):
    """品質報告模型"""
    report_id: str
    period_start: datetime
    period_end: datetime
    summary: Dict[str, Any]
    trends: List[Dict[str, Any]]
    anomalies: List[Dict[str, Any]]
    recommendations: List[str]
    generated_at: datetime

print("✅ Pydantic 驗證模型定義完成")

✅ Pydantic 驗證模型定義完成


## 🔍 多維度品質評估引擎

### 核心評估器實作

In [ ]:
class MultiDimensionalQualityAssessor:
    """
    多維度品質評估引擎
    基於 ISO 25012 標準實作全面的品質評估
    """
    
    def __init__(self, language: str = 'en'):
        """初始化品質評估器"""
        self.language = language
        self.version = "1.0"
        
        # 初始化語言工具
        try:
            self.grammar_tool = language_tool_python.LanguageTool(language)
            print("✅ LanguageTool 初始化完成")
        except Exception as e:
            print(f"⚠️  LanguageTool 初始化失敗: {e}")
            self.grammar_tool = None
        
        # 初始化拼字檢查器
        try:
            self.spell_checker = SpellChecker(language=language)
            print("✅ SpellChecker 初始化完成")
        except Exception as e:
            print(f"⚠️  SpellChecker 初始化失敗: {e}")
            self.spell_checker = None
        
        # 預設品質權重 (基於 ISO 25012)
        self.default_weights = {
            'accuracy': 0.20,
            'completeness': 0.20,
            'consistency': 0.15,
            'currency': 0.15,
            'understandability': 0.15,
            'traceability': 0.15
        }
        
        # 品質閾值
        self.thresholds = {
            'min_word_count': 10,
            'max_sentence_length': 40,
            'min_readability_score': 30,
            'max_spelling_error_rate': 0.05,
            'max_grammar_error_rate': 0.03
        }
        
        print("✅ 多維度品質評估器初始化完成")
    
    def assess_document_quality(self, 
                              content: str, 
                              metadata: Optional[Dict] = None,
                              custom_weights: Optional[Dict] = None) -> QualityMetrics:
        """
        評估文檔的多維度品質
        
        Args:
            content: 文檔內容
            metadata: 文檔元資料
            custom_weights: 自訂權重
            
        Returns:
            QualityMetrics: 品質評估結果
        """
        if not content or not content.strip():
            return QualityMetrics()
        
        metadata = metadata or {}
        weights = custom_weights or self.default_weights
        
        try:
            # 1. 準確性評估
            accuracy = self._assess_accuracy(content)
            
            # 2. 完整性評估
            completeness = self._assess_completeness(content, metadata)
            
            # 3. 一致性評估
            consistency = self._assess_consistency(content)
            
            # 4. 時效性評估
            currency = self._assess_currency(content, metadata)
            
            # 5. 可理解性評估
            understandability = self._assess_understandability(content)
            
            # 6. 可追溯性評估
            traceability = self._assess_traceability(content, metadata)
            
            # 計算綜合品質分數
            overall_score = (
                weights['accuracy'] * accuracy +
                weights['completeness'] * completeness +
                weights['consistency'] * consistency +
                weights['currency'] * currency +
                weights['understandability'] * understandability +
                weights['traceability'] * traceability
            )
            
            return QualityMetrics(
                accuracy=accuracy,
                completeness=completeness,
                consistency=consistency,
                currency=currency,
                understandability=understandability,
                traceability=traceability,
                overall_score=overall_score
            )
            
        except Exception as e:
            logger.error(f"品質評估失敗: {e}")
            return QualityMetrics()
    
    def _assess_accuracy(self, content: str) -> float:
        """
        評估準確性
        基於語法錯誤、拼字錯誤、格式一致性
        """
        accuracy_score = 1.0
        
        # 拼字錯誤檢查
        if self.spell_checker:
            words = re.findall(r'\b\w+\b', content.lower())
            if words:
                misspelled = self.spell_checker.unknown(words)
                spelling_error_rate = len(misspelled) / len(words)
                spelling_penalty = min(spelling_error_rate / self.thresholds['max_spelling_error_rate'], 1.0)
                accuracy_score -= 0.3 * spelling_penalty
        
        # 語法錯誤檢查
        if self.grammar_tool:
            try:
                # 限制檢查長度避免超時
                check_content = content[:2000] if len(content) > 2000 else content
                grammar_errors = self.grammar_tool.check(check_content)
                words_count = len(check_content.split())
                
                if words_count > 0:
                    grammar_error_rate = len(grammar_errors) / words_count
                    grammar_penalty = min(grammar_error_rate / self.thresholds['max_grammar_error_rate'], 1.0)
                    accuracy_score -= 0.4 * grammar_penalty
            except Exception:
                pass  # 忽略語法檢查錯誤
        
        # 格式一致性檢查
        format_consistency = self._check_format_consistency(content)
        accuracy_score -= 0.3 * (1.0 - format_consistency)
        
        return max(0.0, min(1.0, accuracy_score))
    
    def _assess_completeness(self, content: str, metadata: Dict) -> float:
        """
        評估完整性
        基於內容長度、結構完整性、必要元素存在
        """
        completeness_score = 0.0
        
        # 1. 內容長度評估 (40%)
        word_count = len(content.split())
        if word_count >= self.thresholds['min_word_count']:
            # 基於對數函數的評分，避免過度懲罰長文檔
            length_score = min(1.0, np.log10(word_count) / 3.0)
            completeness_score += 0.4 * length_score
        
        # 2. 結構完整性評估 (30%)
        structure_score = self._assess_document_structure(content)
        completeness_score += 0.3 * structure_score
        
        # 3. 元資料完整性評估 (20%)
        metadata_score = self._assess_metadata_completeness(metadata)
        completeness_score += 0.2 * metadata_score
        
        # 4. 內容覆蓋度評估 (10%)
        coverage_score = self._assess_content_coverage(content)
        completeness_score += 0.1 * coverage_score
        
        return max(0.0, min(1.0, completeness_score))
    
    def _assess_consistency(self, content: str) -> float:
        """
        評估一致性
        基於術語使用、格式風格、寫作風格的一致性
        """
        consistency_score = 1.0
        
        # 1. 術語一致性檢查
        terminology_consistency = self._check_terminology_consistency(content)
        consistency_score *= terminology_consistency
        
        # 2. 格式一致性檢查
        format_consistency = self._check_format_consistency(content)
        consistency_score *= format_consistency
        
        # 3. 寫作風格一致性
        style_consistency = self._check_writing_style_consistency(content)
        consistency_score *= style_consistency
        
        return max(0.0, min(1.0, consistency_score))
    
    def _assess_currency(self, content: str, metadata: Dict) -> float:
        """
        評估時效性
        基於文檔年齡、內容新鮮度、引用新穎性
        """
        currency_score = 1.0
        
        # 1. 文檔年齡評估
        if 'created_date' in metadata:
            try:
                created_date = metadata['created_date']
                if isinstance(created_date, str):
                    created_date = datetime.fromisoformat(created_date.replace('Z', '+00:00'))
                
                age_days = (datetime.now() - created_date).days
                
                # 基於文檔類型調整衰減率
                doc_type = metadata.get('document_type', 'unknown')
                if doc_type == 'academic_paper':
                    half_life_days = 1825  # 學術論文5年半衰期
                elif doc_type == 'business_report':
                    half_life_days = 365   # 商業報告1年半衰期
                else:
                    half_life_days = 730   # 預設2年半衰期
                
                # 使用指數衰減模型
                age_factor = np.exp(-0.693 * age_days / half_life_days)
                currency_score *= age_factor
                
            except Exception:
                currency_score *= 0.5  # 無法確定年齡時的懲罰
        
        # 2. 內容新鮮度評估（檢查是否包含最新資訊）
        freshness_score = self._assess_content_freshness(content)
        currency_score = 0.7 * currency_score + 0.3 * freshness_score
        
        return max(0.0, min(1.0, currency_score))
    
    def _assess_understandability(self, content: str) -> float:
        """
        評估可理解性
        基於可讀性指標、句子複雜度、專業術語密度
        """
        understandability_score = 0.0
        
        try:
            # 1. Flesch Reading Ease Score (40%)
            flesch_score = textstat.flesch_reading_ease(content)
            # 將 Flesch 分數 (0-100) 正規化為 0-1
            normalized_flesch = max(0, min(100, flesch_score)) / 100
            understandability_score += 0.4 * normalized_flesch
            
            # 2. 平均句子長度評估 (25%)
            avg_sentence_length = textstat.avg_sentence_length(content)
            # 理想句子長度為15-25字，超出範圍會降低分數
            if 15 <= avg_sentence_length <= 25:
                sentence_score = 1.0
            else:
                deviation = min(abs(avg_sentence_length - 20), 20)
                sentence_score = max(0, 1.0 - deviation / 20)
            understandability_score += 0.25 * sentence_score
            
            # 3. 詞彙複雜度評估 (20%)
            difficult_words_ratio = textstat.difficult_words(content) / max(1, textstat.lexicon_count(content))
            vocabulary_score = max(0, 1.0 - difficult_words_ratio)
            understandability_score += 0.2 * vocabulary_score
            
            # 4. 段落結構評估 (15%)
            paragraph_score = self._assess_paragraph_structure(content)
            understandability_score += 0.15 * paragraph_score
            
        except Exception as e:
            logger.warning(f"可理解性評估部分失敗: {e}")
            understandability_score = 0.5  # 預設中等分數
        
        return max(0.0, min(1.0, understandability_score))
    
    def _assess_traceability(self, content: str, metadata: Dict) -> float:
        """
        評估可追溯性
        基於來源標註、引用完整性、版本資訊
        """
        traceability_score = 0.0
        
        # 1. 元資料追溯性 (40%)
        metadata_trace_score = 0.0
        required_metadata = ['authors', 'created_date', 'source_url', 'version']
        
        for field in required_metadata:
            if field in metadata and metadata[field]:
                metadata_trace_score += 0.25
        
        traceability_score += 0.4 * metadata_trace_score
        
        # 2. 引用和參考文獻 (35%)
        citation_score = self._assess_citations(content)
        traceability_score += 0.35 * citation_score
        
        # 3. 來源標示 (25%)
        source_attribution_score = self._assess_source_attribution(content)
        traceability_score += 0.25 * source_attribution_score
        
        return max(0.0, min(1.0, traceability_score))

print("✅ MultiDimensionalQualityAssessor 核心方法定義完成")

✅ MultiDimensionalQualityAssessor 核心方法定義完成


### 輔助評估方法實作

In [ ]:
# 繼續 MultiDimensionalQualityAssessor 的輔助方法

def _check_format_consistency(self, content: str) -> float:
    """檢查格式一致性"""
    consistency_score = 1.0
    
    # 檢查標題格式一致性
    title_patterns = [
        r'^\d+\.\s+\w+',  # "1. Title"
        r'^#+\s+\w+',      # "# Title" or "## Title"
        r'^[A-Z][^\n]*:$', # "Title:"
    ]
    
    title_matches = []
    for pattern in title_patterns:
        matches = re.findall(pattern, content, re.MULTILINE)
        if matches:
            title_matches.append(len(matches))
    
    # 如果有多種標題格式，降低一致性分數
    if len(title_matches) > 1:
        consistency_score *= 0.8
    
    # 檢查列表格式一致性
    list_patterns = [
        r'^\s*[-*+]\s+',   # bullet points
        r'^\s*\d+\.\s+',   # numbered lists
        r'^\s*[a-z]\)\s+', # lettered lists
    ]
    
    list_format_count = 0
    for pattern in list_patterns:
        if re.search(pattern, content, re.MULTILINE):
            list_format_count += 1
    
    if list_format_count > 1:
        consistency_score *= 0.9
    
    return consistency_score

def _check_terminology_consistency(self, content: str) -> float:
    """檢查術語一致性"""
    # 簡化實作：檢查同義詞的使用一致性
    synonyms_groups = [
        ['AI', 'artificial intelligence', 'machine intelligence'],
        ['ML', 'machine learning'],
        ['DL', 'deep learning'],
        ['NLP', 'natural language processing'],
        ['API', 'application programming interface']
    ]
    
    consistency_penalty = 0.0
    
    for group in synonyms_groups:
        found_variants = []
        for term in group:
            if re.search(r'\b' + re.escape(term) + r'\b', content, re.IGNORECASE):
                found_variants.append(term)
        
        # 如果使用了多種同義詞，扣分
        if len(found_variants) > 1:
            consistency_penalty += 0.05
    
    return max(0.0, 1.0 - consistency_penalty)

def _check_writing_style_consistency(self, content: str) -> float:
    """檢查寫作風格一致性"""
    sentences = re.split(r'[.!?]+', content)
    sentences = [s.strip() for s in sentences if s.strip()]
    
    if len(sentences) < 3:
        return 1.0  # 太短無法評估
    
    # 檢查句子長度變異性
    sentence_lengths = [len(s.split()) for s in sentences]
    length_cv = np.std(sentence_lengths) / (np.mean(sentence_lengths) + 1e-6)
    
    # 變異係數越大，一致性越低
    length_consistency = max(0.0, 1.0 - min(length_cv / 1.5, 1.0))
    
    return length_consistency

def _assess_document_structure(self, content: str) -> float:
    """評估文檔結構完整性"""
    structure_score = 0.0
    
    # 檢查是否有標題
    title_patterns = [r'^.{1,100}$', r'^#+\s+.+$', r'^\d+\.\s+.+$']
    has_titles = any(re.search(pattern, content, re.MULTILINE) for pattern in title_patterns)
    if has_titles:
        structure_score += 0.3
    
    # 檢查是否有段落分隔
    paragraphs = content.split('\n\n')
    if len(paragraphs) > 1:
        structure_score += 0.2
    
    # 檢查是否有列表或要點
    list_pattern = r'^\s*[-*+\d]+[\.)\s]'
    if re.search(list_pattern, content, re.MULTILINE):
        structure_score += 0.2
    
    # 檢查段落長度合理性
    avg_paragraph_length = np.mean([len(p.split()) for p in paragraphs if p.strip()])
    if 20 <= avg_paragraph_length <= 200:
        structure_score += 0.3
    
    return min(1.0, structure_score)

def _assess_metadata_completeness(self, metadata: Dict) -> float:
    """評估元資料完整性"""
    if not metadata:
        return 0.0
    
    # 重要元資料欄位及其權重
    important_fields = {
        'title': 0.25,
        'authors': 0.20,
        'created_date': 0.15,
        'document_type': 0.15,
        'keywords': 0.10,
        'language': 0.05,
        'organization': 0.10
    }
    
    completeness_score = 0.0
    
    for field, weight in important_fields.items():
        if field in metadata and metadata[field]:
            # 檢查值的品質
            value = metadata[field]
            if isinstance(value, str) and len(value.strip()) > 0:
                completeness_score += weight
            elif isinstance(value, list) and len(value) > 0:
                completeness_score += weight
            elif value is not None:
                completeness_score += weight
    
    return completeness_score

def _assess_content_coverage(self, content: str) -> float:
    """評估內容覆蓋度"""
    # 檢查內容是否涵蓋了主要要素
    coverage_indicators = {
        'introduction': [r'\bintroduction\b', r'\boverview\b', r'\bbackground\b'],
        'methodology': [r'\bmethodology\b', r'\bmethod\b', r'\bapproach\b'],
        'results': [r'\bresults\b', r'\bfindings\b', r'\boutcome\b'],
        'conclusion': [r'\bconclusion\b', r'\bsummary\b', r'\bdiscussion\b']
    }
    
    coverage_score = 0.0
    content_lower = content.lower()
    
    for section, patterns in coverage_indicators.items():
        if any(re.search(pattern, content_lower) for pattern in patterns):
            coverage_score += 0.25
    
    return coverage_score

def _assess_content_freshness(self, content: str) -> float:
    """評估內容新鮮度"""
    # 檢查是否包含最近年份的資訊
    current_year = datetime.now().year
    year_pattern = r'\b(20\d{2})\b'
    years_found = [int(year) for year in re.findall(year_pattern, content)]
    
    if not years_found:
        return 0.5  # 沒有年份資訊
    
    # 計算最新年份的新鮮度
    latest_year = max(years_found)
    age = current_year - latest_year
    
    # 3年內為新鮮，之後線性衰減
    if age <= 3:
        return 1.0
    elif age <= 10:
        return 1.0 - (age - 3) / 7
    else:
        return 0.0

def _assess_paragraph_structure(self, content: str) -> float:
    """評估段落結構"""
    paragraphs = [p.strip() for p in content.split('\n\n') if p.strip()]
    
    if not paragraphs:
        return 0.0
    
    # 評估段落長度分佈
    paragraph_lengths = [len(p.split()) for p in paragraphs]
    avg_length = np.mean(paragraph_lengths)
    
    # 理想段落長度為50-150字
    if 50 <= avg_length <= 150:
        length_score = 1.0
    else:
        deviation = min(abs(avg_length - 100), 100)
        length_score = max(0, 1.0 - deviation / 100)
    
    # 評估段落數量合理性
    paragraph_count = len(paragraphs)
    total_words = sum(paragraph_lengths)
    
    if total_words > 0:
        avg_words_per_paragraph = total_words / paragraph_count
        if 30 <= avg_words_per_paragraph <= 200:
            count_score = 1.0
        else:
            count_score = 0.7
    else:
        count_score = 0.0
    
    return 0.6 * length_score + 0.4 * count_score

def _assess_citations(self, content: str) -> float:
    """評估引用品質"""
    # 檢查各種引用格式
    citation_patterns = [
        r'\[\d+\]',                    # [1], [2]
        r'\(\w+\s+et\s+al\.?,?\s+\d{4}\)', # (Smith et al., 2020)
        r'\(\w+,?\s+\d{4}\)',          # (Smith, 2020)
        r'\w+\s+\(\d{4}\)',            # Smith (2020)
        r'doi:\s*10\.\d+',            # DOI references
        r'https?://[\w.-]+',          # URL references
    ]
    
    citation_count = 0
    for pattern in citation_patterns:
        matches = re.findall(pattern, content, re.IGNORECASE)
        citation_count += len(matches)
    
    # 根據文檔長度調整期望的引用數量
    word_count = len(content.split())
    expected_citations = max(1, word_count // 500)  # 每500字期望1個引用
    
    citation_ratio = min(citation_count / expected_citations, 1.0)
    
    return citation_ratio

def _assess_source_attribution(self, content: str) -> float:
    """評估來源標註"""
    attribution_indicators = [
        r'according\s+to\s+\w+',     # "according to Smith"
        r'\w+\s+states?\s+that',      # "Smith states that"
        r'as\s+reported\s+by',       # "as reported by"
        r'source:',                   # "Source:"
        r'from\s+\w+\s+study',       # "from ABC study"
        r'based\s+on\s+\w+',         # "based on research"
    ]
    
    attribution_count = 0
    for pattern in attribution_indicators:
        matches = re.findall(pattern, content, re.IGNORECASE)
        attribution_count += len(matches)
    
    # 標準化分數
    word_count = len(content.split())
    attribution_ratio = min(attribution_count / max(1, word_count // 1000), 1.0)
    
    return attribution_ratio

# 將方法添加到 MultiDimensionalQualityAssessor 類別
MultiDimensionalQualityAssessor._check_format_consistency = _check_format_consistency
MultiDimensionalQualityAssessor._check_terminology_consistency = _check_terminology_consistency
MultiDimensionalQualityAssessor._check_writing_style_consistency = _check_writing_style_consistency
MultiDimensionalQualityAssessor._assess_document_structure = _assess_document_structure
MultiDimensionalQualityAssessor._assess_metadata_completeness = _assess_metadata_completeness
MultiDimensionalQualityAssessor._assess_content_coverage = _assess_content_coverage
MultiDimensionalQualityAssessor._assess_content_freshness = _assess_content_freshness
MultiDimensionalQualityAssessor._assess_paragraph_structure = _assess_paragraph_structure
MultiDimensionalQualityAssessor._assess_citations = _assess_citations
MultiDimensionalQualityAssessor._assess_source_attribution = _assess_source_attribution

print("✅ MultiDimensionalQualityAssessor 輔助方法定義完成")

✅ MultiDimensionalQualityAssessor 輔助方法定義完成


## 📊 異常檢測系統

### 機器學習驅動的異常檢測

In [ ]:
class QualityAnomalyDetector:
    """
    品質異常檢測系統
    使用多種機器學習技術檢測品質指標的異常變化
    """
    
    def __init__(self, config: AnomalyDetectionConfig):
        """初始化異常檢測器"""
        self.config = config
        self.models = {}
        self.scalers = {}
        self.training_history = []
        
        # 初始化檢測模型
        self._initialize_models()
        
        print(f"✅ 品質異常檢測器初始化完成 (方法: {config.method})")
    
    def _initialize_models(self):
        """初始化檢測模型"""
        if self.config.method == 'isolation_forest':
            self.models['primary'] = IsolationForest(
                contamination=self.config.sensitivity,
                random_state=42,
                n_estimators=100
            )
        elif self.config.method == 'one_class_svm':
            self.models['primary'] = OneClassSVM(
                nu=self.config.sensitivity,
                kernel='rbf',
                gamma='scale'
            )
        
        # 資料標準化器
        self.scalers['primary'] = StandardScaler()
    
    def train_detector(self, quality_data: List[Dict]) -> bool:
        """
        訓練異常檢測器
        
        Args:
            quality_data: 歷史品質評估資料
            
        Returns:
            bool: 訓練是否成功
        """
        if len(quality_data) < self.config.min_samples:
            logger.warning(f"訓練資料不足 ({len(quality_data)} < {self.config.min_samples})")
            return False
        
        try:
            # 準備訓練資料
            training_features = self._prepare_features(quality_data)
            
            if training_features.empty:
                logger.error("無法提取訓練特徵")
                return False
            
            # 標準化特徵
            X_scaled = self.scalers['primary'].fit_transform(training_features)
            
            # 訓練主模型
            if self.config.method in ['isolation_forest', 'one_class_svm']:
                self.models['primary'].fit(X_scaled)
            
            # 記錄訓練歷史
            self.training_history.append({
                'timestamp': datetime.now(),
                'sample_count': len(quality_data),
                'feature_count': training_features.shape[1],
                'method': self.config.method,
                'sensitivity': self.config.sensitivity
            })
            
            print(f"✅ 異常檢測器訓練完成 (樣本: {len(quality_data)}, 特徵: {training_features.shape[1]})")
            return True
            
        except Exception as e:
            logger.error(f"異常檢測器訓練失敗: {e}")
            return False
    
    def detect_anomalies(self, new_quality_data: List[Dict]) -> List[Dict]:
        """
        檢測品質異常
        
        Args:
            new_quality_data: 新的品質評估資料
            
        Returns:
            List[Dict]: 檢測到的異常列表
        """
        if not hasattr(self.models['primary'], 'predict'):
            logger.warning("檢測器尚未訓練，無法檢測異常")
            return []
        
        anomalies = []
        
        try:
            # 準備檢測特徵
            detection_features = self._prepare_features(new_quality_data)
            
            if detection_features.empty:
                return anomalies
            
            # 標準化特徵
            X_scaled = self.scalers['primary'].transform(detection_features)
            
            # 執行異常檢測
            if self.config.method in ['isolation_forest', 'one_class_svm']:
                predictions = self.models['primary'].predict(X_scaled)
                anomaly_scores = self.models['primary'].score_samples(X_scaled)
                
                # 處理異常檢測結果
                for i, (prediction, score) in enumerate(zip(predictions, anomaly_scores)):
                    if prediction == -1:  # 異常樣本
                        quality_record = new_quality_data[i]
                        
                        # 分析異常原因
                        anomaly_details = self._analyze_anomaly(
                            quality_record, detection_features.iloc[i], score
                        )
                        
                        anomalies.append({
                            'document_id': quality_record.get('document_id'),
                            'assessment_id': quality_record.get('assessment_id'),
                            'anomaly_score': abs(score),
                            'detection_method': self.config.method,
                            'detected_at': datetime.now(),
                            'details': anomaly_details
                        })
            
            # 統計檢測
            statistical_anomalies = self._detect_statistical_anomalies(new_quality_data)
            anomalies.extend(statistical_anomalies)
            
            print(f"🔍 檢測完成：發現 {len(anomalies)} 個異常")
            return anomalies
            
        except Exception as e:
            logger.error(f"異常檢測失敗: {e}")
            return []
    
    def _prepare_features(self, quality_data: List[Dict]) -> pd.DataFrame:
        """準備特徵向量"""
        features = []
        
        for record in quality_data:
            feature_vector = {
                'accuracy': record.get('accuracy_score', 0.0),
                'completeness': record.get('completeness_score', 0.0),
                'consistency': record.get('consistency_score', 0.0),
                'currency': record.get('currency_score', 0.0),
                'understandability': record.get('understandability_score', 0.0),
                'traceability': record.get('traceability_score', 0.0),
                'overall': record.get('overall_score', 0.0)
            }
            
            # 添加衍生特徵
            if self.config.enable_multivariate:
                # 維度間的比率
                feature_vector['acc_comp_ratio'] = (
                    feature_vector['accuracy'] / max(feature_vector['completeness'], 0.01)
                )
                feature_vector['cons_under_ratio'] = (
                    feature_vector['consistency'] / max(feature_vector['understandability'], 0.01)
                )
                
                # 品質方差
                scores = [
                    feature_vector['accuracy'], feature_vector['completeness'],
                    feature_vector['consistency'], feature_vector['currency'],
                    feature_vector['understandability'], feature_vector['traceability']
                ]
                feature_vector['score_variance'] = np.var(scores)
                feature_vector['score_range'] = max(scores) - min(scores)
            
            features.append(feature_vector)
        
        return pd.DataFrame(features)
    
    def _analyze_anomaly(self, quality_record: Dict, features: pd.Series, anomaly_score: float) -> Dict:
        """分析異常原因"""
        details = {
            'anomaly_type': 'ml_detected',
            'severity': self._calculate_severity(anomaly_score),
            'affected_dimensions': [],
            'explanation': []
        }
        
        # 分析各維度異常
        dimension_scores = {
            'accuracy': features.get('accuracy', 0),
            'completeness': features.get('completeness', 0),
            'consistency': features.get('consistency', 0),
            'currency': features.get('currency', 0),
            'understandability': features.get('understandability', 0),
            'traceability': features.get('traceability', 0)
        }
        
        # 識別異常維度
        for dimension, score in dimension_scores.items():
            if score < 0.3:  # 極低分數
                details['affected_dimensions'].append(dimension)
                details['explanation'].append(f"{dimension} 分數異常偏低 ({score:.3f})")
            elif score > 0.95:  # 異常高分
                details['affected_dimensions'].append(dimension)
                details['explanation'].append(f"{dimension} 分數異常偏高 ({score:.3f})")
        
        # 檢查分數變異性
        if 'score_variance' in features and features['score_variance'] > 0.1:
            details['explanation'].append(f"品質維度間變異性過大 ({features['score_variance']:.3f})")
        
        return details
    
    def _detect_statistical_anomalies(self, quality_data: List[Dict]) -> List[Dict]:
        """統計方法檢測異常"""
        anomalies = []
        
        if len(quality_data) < 3:
            return anomalies
        
        # 提取整體分數
        overall_scores = [record.get('overall_score', 0.0) for record in quality_data]
        
        # Z-Score 檢測
        if len(overall_scores) >= 5:
            mean_score = np.mean(overall_scores)
            std_score = np.std(overall_scores)
            
            if std_score > 0:
                for i, (record, score) in enumerate(zip(quality_data, overall_scores)):
                    z_score = abs((score - mean_score) / std_score)
                    
                    if z_score > 2.5:  # 2.5 標準差以外視為異常
                        anomalies.append({
                            'document_id': record.get('document_id'),
                            'assessment_id': record.get('assessment_id'),
                            'anomaly_score': z_score,
                            'detection_method': 'z_score',
                            'detected_at': datetime.now(),
                            'details': {
                                'anomaly_type': 'statistical',
                                'severity': 'high' if z_score > 3.0 else 'medium',
                                'z_score': z_score,
                                'expected_range': (mean_score - 2*std_score, mean_score + 2*std_score),
                                'actual_value': score
                            }
                        })
        
        return anomalies
    
    def _calculate_severity(self, anomaly_score: float) -> str:
        """計算異常嚴重程度"""
        abs_score = abs(anomaly_score)
        
        if abs_score > 0.8:
            return 'critical'
        elif abs_score > 0.6:
            return 'high'
        elif abs_score > 0.4:
            return 'medium'
        else:
            return 'low'
    
    def get_detector_stats(self) -> Dict:
        """獲取檢測器統計資訊"""
        return {
            'method': self.config.method,
            'sensitivity': self.config.sensitivity,
            'min_samples': self.config.min_samples,
            'training_history_count': len(self.training_history),
            'last_training': self.training_history[-1] if self.training_history else None,
            'is_trained': hasattr(self.models.get('primary', {}), 'predict')
        }

print("✅ QualityAnomalyDetector 定義完成")

✅ QualityAnomalyDetector 定義完成


## 🧪 完整測試與驗證

### 建立測試環境

In [ ]:
# 設定測試環境
test_dir = PROJECT_ROOT / 'notebooks' / '03_quality_control' / 'test_data'
test_dir.mkdir(parents=True, exist_ok=True)

print(f"📂 測試目錄: {test_dir}")

# 建立更多樣化的測試文檔（8個文檔確保足夠樣本）
test_documents = {
    'high_quality_paper.txt': """
A Comprehensive Analysis of Machine Learning Applications in Healthcare

Abstract
This study presents a thorough examination of machine learning applications in healthcare systems. We analyze over 500 implementations across various medical domains and provide evidence-based recommendations for future deployment. Our findings indicate a 35% improvement in diagnostic accuracy when ML systems are properly integrated.

1. Introduction
Machine learning has emerged as a transformative technology in healthcare, offering unprecedented opportunities for improving patient outcomes. According to recent studies (Smith et al., 2023), the integration of AI systems has shown remarkable success in diagnostic imaging, drug discovery, and personalized medicine.

The healthcare industry generates approximately 2.5 quintillion bytes of data daily (Johnson, 2023). This vast amount of information presents both opportunities and challenges for healthcare providers seeking to leverage data-driven insights.

2. Methodology
We conducted a systematic literature review of peer-reviewed publications from 2020 to 2023. Our search strategy included multiple databases: PubMed, IEEE Xplore, and ACM Digital Library. We identified 847 relevant studies and selected 156 for detailed analysis based on our inclusion criteria.

3. Results
Our analysis reveals three primary categories of ML applications: diagnostic systems (45%), predictive analytics (35%), and treatment optimization (20%). Diagnostic systems showed the highest success rates, with average accuracy improvements of 28% compared to traditional methods.

4. Discussion
The implementation of machine learning in healthcare requires careful consideration of ethical implications, data privacy, and regulatory compliance. Our findings suggest that successful deployments require interdisciplinary collaboration between technologists and healthcare professionals.

5. Conclusion
Machine learning represents a significant advancement in healthcare technology. With proper implementation and ongoing evaluation, these systems can substantially improve patient care quality and operational efficiency.

References
[1] Smith, J., et al. (2023). "AI in Medical Diagnosis." Nature Medicine, 15(3), 245-267.
[2] Johnson, M. (2023). "Healthcare Data Analytics." JAMA, 180(12), 1234-1245.
""",
    
    'medium_quality_report.txt': """
Quarterly Business Review Q3 2024

Executive Summary
This report covers our business performance for Q3 2024. Revenue was good and we met most targets. There were some challenges but overall things went well.

Financial Results
Revenue was $2.5M which is up from last quarter. Costs were controlled and we made a profit. The sales team did well and we got new customers.

Operations
Operations ran smoothly most of the time. We had some issues with the new system but they were resolved quickly. Staff worked hard and productivity was ok.

Marketing
Marketing campaigns performed adequately. We got some leads and conversion rates were reasonable. Social media engagement was standard.

Recommendations
We should continue current strategies and maybe improve some processes. Training might help and we could look at new technologies.
""",
    
    'low_quality_text.txt': """
AI stuff

AI is good. It does things. Machines learn stuff and then they know things. This is useful for business and other things too.

There are different kinds of AI like ML and DL and NLP. They all do different stuff but its all related somehow.

Companies use AI for various reasons. It helps with efficiency and productivity and other metrics. ROI is important.

The future looks bright for AI technology. More companies will adopt it. Training is needed. Ethics are important too but progress shouldnt be slowed down.

In conclusion AI is the future and companies should invest in it soon before competitors get ahead.
""",
    
    'technical_documentation.txt': """
API Documentation - User Authentication Service v2.1

Overview
The User Authentication Service provides secure authentication and authorization capabilities for applications. This service implements OAuth 2.0 and JWT token standards with enterprise-grade security features.

Authentication Endpoints

POST /auth/login
Description: Authenticates a user with email/password credentials
Parameters:
- email (string, required): User's email address
- password (string, required): User's password
- remember_me (boolean, optional): Extended session duration

Response:
{
  "access_token": "eyJhbGciOiJIUzI1NiIs...",
  "refresh_token": "dGhpcyBpcyBhIHJlZnJlc2g...",
  "expires_in": 3600,
  "user_id": "12345"
}

Error Codes:
- 400: Invalid credentials
- 401: Account locked
- 429: Rate limit exceeded

POST /auth/refresh
Description: Refreshes an expired access token using a valid refresh token
Headers:
- Authorization: Bearer {refresh_token}

Security Considerations
- All endpoints require HTTPS in production
- Rate limiting: 5 requests per minute per IP
- Token lifetime: 1 hour (access), 30 days (refresh)
- Failed login attempts are logged for security monitoring

Implementation Example
```python
import requests

response = requests.post('https://api.example.com/auth/login', {
    'email': 'user@example.com',
    'password': 'secure_password123'
})

if response.status_code == 200:
    tokens = response.json()
    access_token = tokens['access_token']
```

Version History
- v2.1 (2024-01-15): Added refresh token rotation
- v2.0 (2023-12-01): OAuth 2.0 compliance updates
- v1.5 (2023-10-15): Enhanced security features
""",
    
    'policy_document.txt': """
Data Privacy and Security Policy

Document ID: POL-2024-001
Effective Date: January 1, 2024
Last Reviewed: October 15, 2024
Next Review: January 1, 2025

1. Purpose and Scope
This policy establishes the framework for protecting personal and sensitive data within our organization. It applies to all employees, contractors, and third-party vendors who handle, process, or have access to organizational data.

2. Policy Statement
Our organization is committed to protecting the privacy and security of all data entrusted to us. We will implement appropriate technical, administrative, and physical safeguards to ensure data confidentiality, integrity, and availability.

3. Data Classification
3.1 Public Data: Information that can be freely shared without risk
3.2 Internal Data: Information restricted to authorized personnel
3.3 Confidential Data: Sensitive information requiring special protection
3.4 Restricted Data: Highly sensitive information with strict access controls

4. Access Controls
- Role-based access control (RBAC) must be implemented
- Principle of least privilege applies to all data access
- Regular access reviews conducted quarterly
- Multi-factor authentication required for sensitive systems

5. Data Retention
- Personal data retained only as long as necessary for business purposes
- Retention schedules established for each data category
- Secure deletion procedures implemented for expired data

6. Incident Response
- Data breaches reported within 72 hours to relevant authorities
- Incident response team activated for all security incidents
- Post-incident analysis conducted to prevent recurrence

7. Training and Awareness
- Annual privacy training required for all staff
- Regular security awareness communications
- Specialized training for data handlers

8. Compliance
This policy ensures compliance with GDPR, CCPA, and other applicable regulations.

Approved by: Chief Information Security Officer
Date: January 1, 2024
""",
    
    'research_proposal.txt': """
Research Proposal: Quantum Computing Applications in Cryptography

Principal Investigator: Dr. Elena Rodriguez
Co-Investigators: Prof. James Chen, Dr. Sarah Kim
Institution: University of Technology
Funding Agency: National Science Foundation
Requested Amount: $2,500,000
Project Duration: 3 years

Abstract
This research proposes to investigate the potential applications of quantum computing in modern cryptographic systems. As quantum computers become more powerful, traditional encryption methods face unprecedented challenges. Our research aims to develop quantum-resistant cryptographic algorithms and explore new quantum cryptographic protocols.

1. Background and Significance
The advent of quantum computing presents both opportunities and threats to cybersecurity. While quantum computers could break existing encryption methods, they also enable new forms of quantum cryptography that offer theoretically unbreakable security.

Current RSA and elliptic curve cryptographic systems rely on the computational difficulty of factoring large integers. Shor's algorithm, when implemented on a sufficiently powerful quantum computer, could solve these problems exponentially faster than classical computers.

2. Research Objectives
2.1 Develop post-quantum cryptographic algorithms resistant to quantum attacks
2.2 Investigate quantum key distribution protocols for secure communication
2.3 Analyze the security implications of hybrid classical-quantum systems
2.4 Create a framework for transitioning from classical to quantum-safe cryptography

3. Methodology
We will employ both theoretical analysis and experimental validation using quantum simulators. Our approach includes:
- Mathematical analysis of quantum algorithms
- Security proofs for proposed protocols
- Implementation and testing on quantum hardware simulators
- Performance comparison with classical methods

4. Expected Outcomes
- 3-5 peer-reviewed publications in top-tier journals
- Development of practical quantum-resistant algorithms
- Training of 4 graduate students and 2 postdoctoral researchers
- Open-source software library for quantum cryptographic protocols

5. Budget Justification
Year 1: $900,000 (personnel: $600,000, equipment: $200,000, travel: $50,000, indirect: $50,000)
Year 2: $850,000 (personnel: $650,000, supplies: $100,000, travel: $50,000, indirect: $50,000)
Year 3: $750,000 (personnel: $600,000, publication: $50,000, travel: $50,000, indirect: $50,000)

References
[1] Shor, P. W. (1994). "Algorithms for quantum computation." Proc. 35th Annual Symposium on Foundations of Computer Science.
[2] Bennett, C. H., & Brassard, G. (1984). "Quantum cryptography." IEEE International Conference on Computers, Systems and Signal Processing.
""",
    
    'user_manual.txt': """
SmartHome Controller User Manual

Model: SH-2024-Pro
Version: 3.2.1
Publication Date: March 2024

Table of Contents
1. Getting Started
2. Initial Setup
3. Device Configuration
4. Advanced Features
5. Troubleshooting
6. Technical Specifications

Chapter 1: Getting Started

Welcome to your new SmartHome Controller! This device allows you to control and monitor all connected smart devices in your home from a single interface.

What's in the Box:
- SmartHome Controller unit
- Power adapter (12V, 2A)
- Ethernet cable
- Quick start guide
- Warranty information

Safety Precautions:
- Do not expose the device to moisture or extreme temperatures
- Use only the provided power adapter
- Keep the device away from direct sunlight
- Ensure proper ventilation around the device

Chapter 2: Initial Setup

Step 1: Physical Installation
1. Choose a central location in your home for optimal wireless coverage
2. Connect the power adapter to the device and plug into a wall outlet
3. Connect the Ethernet cable to your router (optional but recommended)
4. Wait for the LED indicator to turn solid green (approximately 2 minutes)

Step 2: Network Configuration
1. Download the SmartHome app from the App Store or Google Play
2. Create a new account or sign in to your existing account
3. Tap "Add Device" and select "SmartHome Controller"
4. Follow the on-screen instructions to connect to your Wi-Fi network

Chapter 3: Device Configuration

Adding Smart Devices:
The controller supports over 200 different smart device types including:
- Smart lights and switches
- Thermostats and climate controls
- Security cameras and sensors
- Smart locks and garage door openers
- Entertainment systems

To add a device:
1. Open the SmartHome app
2. Tap the "+" button in the top right corner
3. Select the device type from the list
4. Follow the specific pairing instructions for your device

Chapter 4: Advanced Features

Scene Creation:
Create custom scenes to control multiple devices with a single command.

Automation Rules:
Set up rules to trigger actions based on time, device status, or sensor readings.

Voice Control:
Compatible with Amazon Alexa, Google Assistant, and Apple HomeKit.

Chapter 5: Troubleshooting

Common Issues:
Q: Device won't connect to Wi-Fi
A: Ensure you're using a 2.4GHz network and the password is correct

Q: Smart devices not responding
A: Check if the devices have power and are within range of the controller

Q: App crashes or freezes
A: Force close the app and restart it. If the problem persists, reinstall the app.

Technical Support:
Email: support@smarthome.com
Phone: 1-800-SMART-HOME (1-800-762-7846)
Hours: Monday-Friday, 8 AM - 8 PM EST

Chapter 6: Technical Specifications

- Processor: ARM Cortex-A53 Quad-core 1.4GHz
- Memory: 2GB RAM, 16GB storage
- Connectivity: Wi-Fi 802.11 a/b/g/n/ac, Bluetooth 5.0, Zigbee 3.0, Z-Wave Plus
- Operating Temperature: 32°F to 95°F (0°C to 35°C)
- Dimensions: 6" x 4" x 1.5" (152mm x 102mm x 38mm)
- Power Consumption: 12W maximum
""",
    
    'financial_analysis.txt': """
Quarterly Financial Analysis - Q3 2024
TechCorp Industries

Executive Summary
TechCorp Industries demonstrated strong financial performance in Q3 2024, with revenue growth of 18% year-over-year and improved profit margins across all business segments. This analysis examines key financial metrics, market trends, and strategic implications for the remainder of 2024.

Revenue Analysis
Total revenue for Q3 2024 reached $127.3 million, representing an 18% increase compared to Q3 2023 ($107.8 million) and a 12% increase from Q2 2024 ($113.7 million).

Revenue by Segment:
- Software Solutions: $78.2M (61% of total revenue, +22% YoY)
- Hardware Products: $31.4M (25% of total revenue, +8% YoY)
- Professional Services: $17.7M (14% of total revenue, +15% YoY)

The software segment continues to drive growth, benefiting from increased demand for cloud-based solutions and subscription services. Hardware revenue growth was modest due to supply chain constraints, while professional services showed steady improvement.

Profitability Metrics
Gross profit margin improved to 64.2% in Q3 2024, up from 61.8% in Q3 2023. This improvement reflects operational efficiencies and favorable product mix shifts toward higher-margin software offerings.

Operating expenses increased 12% year-over-year to $52.6 million, primarily due to:
- Research and development investments: $23.1M (+15% YoY)
- Sales and marketing: $18.8M (+10% YoY)  
- General and administrative: $10.7M (+8% YoY)

Net income reached $28.4 million, or $1.42 per diluted share, compared to $22.1 million, or $1.12 per diluted share in Q3 2023.

Cash Flow and Balance Sheet
Operating cash flow was $35.7 million in Q3 2024, compared to $28.9 million in Q3 2023. The company ended the quarter with $156.2 million in cash and cash equivalents.

Key balance sheet highlights:
- Total assets: $487.3M (+8% from Q2 2024)
- Total debt: $89.4M (debt-to-equity ratio of 0.23)
- Shareholders' equity: $389.1M

Market Position and Competition
TechCorp maintains a strong competitive position in its core markets. Market share in the enterprise software segment increased to 12.3%, up from 11.7% in the previous year. The company's customer retention rate remains high at 94%.

Key competitive advantages include:
- Proprietary technology platform with high switching costs
- Strong customer relationships and brand recognition
- Comprehensive product portfolio addressing multiple market segments

Risk Factors and Challenges
Several factors could impact future performance:
- Economic uncertainty affecting enterprise technology spending
- Intensifying competition from both established players and startups
- Cybersecurity threats and data privacy regulations
- Talent acquisition and retention in competitive labor markets

Outlook and Guidance
Based on current market conditions and pipeline visibility, management reaffirms full-year 2024 guidance:
- Revenue: $495M - $505M (previous guidance: $490M - $510M)
- Operating margin: 21% - 23%
- Diluted EPS: $5.40 - $5.60

For Q4 2024, the company expects:
- Revenue: $130M - $135M
- Operating margin: 22% - 24%

Strategic Initiatives
Key strategic priorities for the remainder of 2024 include:
1. Accelerating cloud migration for existing customers
2. Expanding international presence in European markets
3. Investing in artificial intelligence and machine learning capabilities
4. Pursuing strategic acquisitions to enhance product portfolio

Conclusion
TechCorp's Q3 2024 performance demonstrates the effectiveness of its strategic focus on high-margin software solutions and operational excellence. The company is well-positioned for continued growth, though management remains cautious about macroeconomic headwinds.

Analyst: Jennifer Walsh, CFA
Date: October 28, 2024
"""
}

# 建立測試檔案
for filename, content in test_documents.items():
    file_path = test_dir / filename
    with open(file_path, 'w', encoding='utf-8') as f:
        f.write(content)

print(f"✅ 建立了 {len(test_documents)} 個測試文檔")

# 準備測試元資料
test_metadata = {
    'high_quality_paper.txt': {
        'title': 'A Comprehensive Analysis of Machine Learning Applications in Healthcare',
        'authors': ['Dr. Sarah Chen', 'Prof. Michael Rodriguez'],
        'created_date': '2024-01-15T10:30:00Z',
        'document_type': 'academic_paper',
        'organization': 'Stanford Medical School',
        'keywords': ['machine learning', 'healthcare', 'diagnostic systems'],
        'version': '1.0'
    },
    'medium_quality_report.txt': {
        'title': 'Quarterly Business Review Q3 2024',
        'authors': ['Business Team'],
        'created_date': '2024-10-01T09:00:00Z',
        'document_type': 'business_report',
        'organization': 'Tech Corp',
        'keywords': ['quarterly review', 'business performance']
    },
    'low_quality_text.txt': {
        'title': 'AI stuff',
        'created_date': '2024-11-01T14:30:00Z',
        'document_type': 'other'
    },
    'technical_documentation.txt': {
        'title': 'API Documentation - User Authentication Service v2.1',
        'authors': ['DevOps Team'],
        'created_date': '2024-01-15T16:45:00Z',
        'document_type': 'technical_doc',
        'organization': 'Engineering Dept',
        'keywords': ['API', 'authentication', 'OAuth'],
        'version': '2.1'
    },
    'policy_document.txt': {
        'title': 'Data Privacy and Security Policy',
        'authors': ['Legal Department', 'IT Security Team'],
        'created_date': '2024-01-01T00:00:00Z',
        'document_type': 'policy',
        'organization': 'Corporate',
        'keywords': ['data privacy', 'security', 'GDPR', 'compliance'],
        'version': 'POL-2024-001'
    },
    'research_proposal.txt': {
        'title': 'Research Proposal: Quantum Computing Applications in Cryptography',
        'authors': ['Dr. Elena Rodriguez', 'Prof. James Chen', 'Dr. Sarah Kim'],
        'created_date': '2024-02-15T09:30:00Z',
        'document_type': 'academic_paper',
        'organization': 'University of Technology',
        'keywords': ['quantum computing', 'cryptography', 'research'],
        'version': '1.0'
    },
    'user_manual.txt': {
        'title': 'SmartHome Controller User Manual',
        'authors': ['Technical Writing Team'],
        'created_date': '2024-03-01T10:00:00Z',
        'document_type': 'technical_doc',
        'organization': 'SmartHome Inc',
        'keywords': ['user manual', 'smart home', 'installation'],
        'version': '3.2.1'
    },
    'financial_analysis.txt': {
        'title': 'Quarterly Financial Analysis - Q3 2024',
        'authors': ['Jennifer Walsh'],
        'created_date': '2024-10-28T14:00:00Z',
        'document_type': 'business_report',
        'organization': 'TechCorp Industries',
        'keywords': ['financial analysis', 'quarterly report', 'revenue'],
        'version': 'Q3-2024'
    }
}

print("✅ 測試元資料準備完成")

### 執行品質評估測試

In [ ]:
print("🔄 開始品質評估測試\n")

# 初始化品質評估器
quality_assessor = MultiDimensionalQualityAssessor()

# 評估所有測試文檔
assessment_results = []

for filename in test_documents.keys():
    file_path = test_dir / filename
    
    # 讀取文檔內容
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()
    
    # 獲取元資料
    metadata = test_metadata.get(filename, {})
    
    # 執行品質評估
    print(f"📊 評估文檔: {filename}")
    quality_metrics = quality_assessor.assess_document_quality(content, metadata)
    
    # 記錄結果
    result = {
        'filename': filename,
        'document_type': metadata.get('document_type', 'unknown'),
        'word_count': len(content.split()),
        'quality_metrics': quality_metrics,
        'quality_status': quality_metrics.get_status().value,
        'assessment_time': datetime.now()
    }
    
    assessment_results.append(result)
    
    # 顯示評估結果
    print(f"  品質狀態: {result['quality_status'].upper()}")
    print(f"  綜合分數: {quality_metrics.overall_score:.3f}")
    print(f"  準確性: {quality_metrics.accuracy:.3f}")
    print(f"  完整性: {quality_metrics.completeness:.3f}")
    print(f"  一致性: {quality_metrics.consistency:.3f}")
    print(f"  時效性: {quality_metrics.currency:.3f}")
    print(f"  可理解性: {quality_metrics.understandability:.3f}")
    print(f"  可追溯性: {quality_metrics.traceability:.3f}")
    print()

print("\n" + "="*60)
print("📊 品質評估摘要")

# 建立評估摘要
summary_data = []
for result in assessment_results:
    metrics = result['quality_metrics']
    summary_data.append({
        '文檔': result['filename'].replace('.txt', ''),
        '類型': result['document_type'],
        '字數': result['word_count'],
        '綜合分數': f"{metrics.overall_score:.3f}",
        '狀態': result['quality_status'],
        '準確性': f"{metrics.accuracy:.2f}",
        '完整性': f"{metrics.completeness:.2f}",
        '一致性': f"{metrics.consistency:.2f}",
        '時效性': f"{metrics.currency:.2f}",
        '可理解性': f"{metrics.understandability:.2f}",
        '可追溯性': f"{metrics.traceability:.2f}"
    })

summary_df = pd.DataFrame(summary_data)
print(summary_df.to_string(index=False))

# 統計分析
overall_scores = [result['quality_metrics'].overall_score for result in assessment_results]
print(f"\n📈 統計分析:")
print(f"平均品質分數: {np.mean(overall_scores):.3f}")
print(f"標準差: {np.std(overall_scores):.3f}")
print(f"最高分數: {max(overall_scores):.3f}")
print(f"最低分數: {min(overall_scores):.3f}")

# 品質狀態分佈
status_counts = {}
for result in assessment_results:
    status = result['quality_status']
    status_counts[status] = status_counts.get(status, 0) + 1

print(f"\n🎯 品質狀態分佈:")
for status, count in status_counts.items():
    print(f"{status.capitalize()}: {count} 個文檔")

🔄 開始品質評估測試

⚠️  LanguageTool 初始化失敗: Detected java 11.0. LanguageTool requires Java >= 17 for version latest.
✅ SpellChecker 初始化完成
✅ 多維度品質評估器初始化完成
📊 評估文檔: high_quality_paper.txt
  品質狀態: GOOD
  綜合分數: 0.713
  準確性: 0.843
  完整性: 0.920
  一致性: 0.468
  時效性: 0.650
  可理解性: 0.381
  可追溯性: 0.900

📊 評估文檔: medium_quality_report.txt
  品質狀態: FAIR
  綜合分數: 0.660
  準確性: 0.908
  完整性: 0.762
  一致性: 0.776
  時效性: 0.650
  可理解性: 0.544
  可追溯性: 0.200

📊 評估文檔: low_quality_text.txt
  品質狀態: FAIR
  綜合分數: 0.537
  準確性: 0.720
  完整性: 0.556
  一致性: 0.668
  時效性: 0.500
  可理解性: 0.610
  可追溯性: 0.100

📊 評估文檔: technical_documentation.txt
  品質狀態: FAIR
  綜合分數: 0.527
  準確性: 0.700
  完整性: 0.729
  一致性: 0.000
  時效性: 0.650
  可理解性: 0.306
  可追溯性: 0.650


📊 品質評估摘要
                     文檔              類型  字數  綜合分數   狀態  準確性  完整性  一致性  時效性 可理解性 可追溯性
     high_quality_paper  academic_paper 298 0.713 good 0.84 0.92 0.47 0.65 0.38 0.90
  medium_quality_report business_report 130 0.660 fair 0.91 0.76 0.78 0.65 0.54 0.20
       low_quality_text    

### 品質評估結果視覺化

In [ ]:
print("📊 生成品質評估視覺化圖表\n")

# 準備視覺化資料
doc_names = [result['filename'].replace('.txt', '').replace('_', ' ') for result in assessment_results]
quality_metrics_data = []

dimensions = ['accuracy', 'completeness', 'consistency', 'currency', 'understandability', 'traceability']
dimension_names = ['準確性', '完整性', '一致性', '時效性', '可理解性', '可追溯性']

for result in assessment_results:
    metrics = result['quality_metrics']
    quality_metrics_data.append([
        metrics.accuracy,
        metrics.completeness,
        metrics.consistency,
        metrics.currency,
        metrics.understandability,
        metrics.traceability
    ])

# 1. 雷達圖 - 多維度品質比較
fig = go.Figure()

colors = ['blue', 'red', 'green', 'orange']
for i, (doc_name, metrics_values) in enumerate(zip(doc_names, quality_metrics_data)):
    fig.add_trace(go.Scatterpolar(
        r=metrics_values + [metrics_values[0]],  # 閉合雷達圖
        theta=dimension_names + [dimension_names[0]],
        fill='toself',
        name=doc_name,
        line_color=colors[i % len(colors)]
    ))

fig.update_layout(
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[0, 1]
        )
    ),
    showlegend=True,
    title="文檔品質多維度雷達圖",
    height=600
)

fig.show()

# 2. 堆疊柱狀圖 - 各維度貢獻
fig2 = go.Figure()

for i, dim_name in enumerate(dimension_names):
    dim_values = [metrics_data[i] for metrics_data in quality_metrics_data]
    
    fig2.add_trace(go.Bar(
        name=dim_name,
        x=doc_names,
        y=dim_values,
        text=[f"{val:.2f}" for val in dim_values],
        textposition='inside'
    ))

fig2.update_layout(
    barmode='group',
    title='各文檔品質維度分數比較',
    xaxis_title='文檔',
    yaxis_title='品質分數',
    height=500,
    xaxis_tickangle=-45
)

fig2.show()

# 3. 綜合品質分數趨勢
overall_scores = [result['quality_metrics'].overall_score for result in assessment_results]
quality_statuses = [result['quality_status'] for result in assessment_results]

# 為不同品質狀態設定顏色
status_colors = {
    'excellent': 'green',
    'good': 'blue', 
    'fair': 'orange',
    'poor': 'red'
}

colors_mapped = [status_colors.get(status, 'gray') for status in quality_statuses]

fig3 = go.Figure()

fig3.add_trace(go.Bar(
    x=doc_names,
    y=overall_scores,
    marker_color=colors_mapped,
    text=[f"{score:.3f}<br>{status}" for score, status in zip(overall_scores, quality_statuses)],
    textposition='outside'
))

# 添加品質閾值線
fig3.add_hline(y=0.9, line_dash="dash", line_color="green", annotation_text="優秀 (>0.9)")
fig3.add_hline(y=0.7, line_dash="dash", line_color="blue", annotation_text="良好 (>0.7)")
fig3.add_hline(y=0.5, line_dash="dash", line_color="orange", annotation_text="普通 (>0.5)")

fig3.update_layout(
    title='文檔綜合品質分數與狀態',
    xaxis_title='文檔',
    yaxis_title='綜合品質分數',
    height=500,
    xaxis_tickangle=-45,
    yaxis=dict(range=[0, 1])
)

fig3.show()

print("✅ 品質評估視覺化圖表生成完成")

📊 生成品質評估視覺化圖表



✅ 品質評估視覺化圖表生成完成


### 測試異常檢測系統

In [ ]:
print("🔍 測試異常檢測系統\n")

# 配置異常檢測器
anomaly_config = AnomalyDetectionConfig(
    method='isolation_forest',
    sensitivity=0.15,
    min_samples=5,  # 現在有8個真實文檔，滿足最小樣本要求
    lookback_days=30,
    enable_multivariate=True
)

# 初始化異常檢測器
anomaly_detector = QualityAnomalyDetector(anomaly_config)

# 準備訓練資料（從8個評估結果轉換）
training_data = []
for result in assessment_results:
    metrics = result['quality_metrics']
    training_record = {
        'document_id': f"doc_{result['filename']}",
        'assessment_id': f"assess_{int(datetime.now().timestamp())}_{result['filename']}",
        'accuracy_score': metrics.accuracy,
        'completeness_score': metrics.completeness,
        'consistency_score': metrics.consistency,
        'currency_score': metrics.currency,
        'understandability_score': metrics.understandability,
        'traceability_score': metrics.traceability,
        'overall_score': metrics.overall_score,
        'assessed_at': result['assessment_time']
    }
    training_data.append(training_record)

print(f"📊 準備了 {len(training_data)} 個真實訓練樣本")

# 訓練異常檢測器
print("🔄 訓練異常檢測器...")
training_success = anomaly_detector.train_detector(training_data)

if training_success:
    print("✅ 異常檢測器訓練成功\n")
    
    # 建立異常測試資料
    print("📊 建立異常測試資料...")
    
    # 創建一些異常的品質記錄
    anomalous_data = [
        # 極低品質文檔
        {
            'document_id': 'anomaly_doc_1',
            'assessment_id': 'assess_anomaly_1',
            'accuracy_score': 0.1,  # 極低
            'completeness_score': 0.2,
            'consistency_score': 0.15,
            'currency_score': 0.3,
            'understandability_score': 0.1,
            'traceability_score': 0.0,
            'overall_score': 0.15,
            'assessed_at': datetime.now()
        },
        # 某個維度異常高
        {
            'document_id': 'anomaly_doc_2',
            'assessment_id': 'assess_anomaly_2', 
            'accuracy_score': 1.0,  # 異常完美
            'completeness_score': 0.98,
            'consistency_score': 1.0,
            'currency_score': 0.95,
            'understandability_score': 1.0,
            'traceability_score': 1.0,
            'overall_score': 0.985,
            'assessed_at': datetime.now()
        },
        # 維度間差異極大
        {
            'document_id': 'anomaly_doc_3',
            'assessment_id': 'assess_anomaly_3',
            'accuracy_score': 0.9,
            'completeness_score': 0.1,  # 極低
            'consistency_score': 0.85,
            'currency_score': 0.02,   # 極低
            'understandability_score': 0.75,
            'traceability_score': 0.8,
            'overall_score': 0.57,
            'assessed_at': datetime.now()
        },
        # 異常低的所有維度
        {
            'document_id': 'anomaly_doc_4',
            'assessment_id': 'assess_anomaly_4',
            'accuracy_score': 0.05,
            'completeness_score': 0.08,
            'consistency_score': 0.03,
            'currency_score': 0.12,
            'understandability_score': 0.06,
            'traceability_score': 0.02,
            'overall_score': 0.06,
            'assessed_at': datetime.now()
        }
    ]
    
    # 加入正常資料作為對照（使用真實評估結果的子集）
    test_data = training_data.copy() + anomalous_data
    
    # 執行異常檢測
    print("🔍 執行異常檢測...")
    detected_anomalies = anomaly_detector.detect_anomalies(test_data)
    
    print(f"\n📊 異常檢測結果: 發現 {len(detected_anomalies)} 個異常\n")
    
    # 顯示檢測到的異常
    for i, anomaly in enumerate(detected_anomalies, 1):
        print(f"異常 {i}:")
        print(f"  文檔ID: {anomaly.get('document_id')}")
        print(f"  異常分數: {anomaly.get('anomaly_score', 0):.3f}")
        print(f"  檢測方法: {anomaly.get('detection_method')}")
        print(f"  檢測時間: {anomaly.get('detected_at')}")
        
        details = anomaly.get('details', {})
        if details:
            print(f"  嚴重程度: {details.get('severity', 'unknown')}")
            if 'affected_dimensions' in details:
                print(f"  影響維度: {', '.join(details['affected_dimensions'])}")
            if 'explanation' in details:
                for explanation in details['explanation']:
                    print(f"  說明: {explanation}")
        print()
    
    # 異常檢測效果分析
    print("📈 異常檢測效果分析:")
    
    # 統計真實文檔vs異常樣本的檢測結果
    real_doc_anomalies = [a for a in detected_anomalies if a.get('document_id', '').startswith('doc_')]
    synthetic_anomalies = [a for a in detected_anomalies if a.get('document_id', '').startswith('anomaly_')]
    
    print(f"  真實文檔中檢測到異常: {len(real_doc_anomalies)} 個")
    print(f"  合成異常樣本中檢測到: {len(synthetic_anomalies)} 個 (共4個異常樣本)")
    print(f"  異常檢測準確率: {len(synthetic_anomalies)/4*100:.1f}%")
    
    if real_doc_anomalies:
        print(f"  真實文檔異常詳情:")
        for anomaly in real_doc_anomalies:
            doc_id = anomaly.get('document_id', '').replace('doc_', '').replace('.txt', '')
            print(f"    - {doc_id}: 分數 {anomaly.get('anomaly_score', 0):.3f}")
    
    # 檢測器統計資訊
    detector_stats = anomaly_detector.get_detector_stats()
    print("\n📈 檢測器統計資訊:")
    for key, value in detector_stats.items():
        print(f"  {key}: {value}")
        
else:
    print("❌ 異常檢測器訓練失敗")
    print(f"  原因: 需要至少 {anomaly_config.min_samples} 個樣本，但只有 {len(training_data)} 個")

print("\n🎉 品質控制系統測試完成！")

## 🎓 學習總結與下一步

### 模組 3 學習成果檢核

完成本模組後，您已經掌握了：

✅ **多維度品質評估框架**
- 基於 ISO 25012 標準的六大品質維度
- 準確性、完整性、一致性評估算法
- 時效性、可理解性、可追溯性評估
- 加權綜合評分機制

✅ **自動化品質監控系統**
- 即時品質評估與狀態分類
- 品質趨勢分析與歷史追蹤
- 多層次品質閾值管理
- 品質報告自動生成

✅ **智能異常檢測技術**
- 統計異常檢測（Z-Score）
- 機器學習異常檢測（Isolation Forest、One-Class SVM）
- 多變量異常檢測
- 異常嚴重程度評估與分類

✅ **品質治理最佳實踐**
- 品質規則引擎設計
- 品質評估結果視覺化
- 異常處理工作流程
- 持續改進機制

### 關鍵技術亮點

1. **全面的品質模型**: 涵蓋文本品質的各個層面
2. **智能化評估**: 結合 NLP 技術和統計方法
3. **可擴展架構**: 支持自訂品質規則和權重
4. **實時監控**: 即時檢測品質變化和異常
5. **可視化分析**: 直觀展示品質趨勢和問題

### 實際應用場景

- **企業文檔管理**: 確保內部文檔的一致性和品質
- **學術論文審查**: 自動化的論文品質初審
- **內容品質監控**: 網站內容、產品文檔品質把關
- **合規性檢查**: 文檔是否符合行業標準和法規要求

### 下一步學習建議

準備進入 **模組 4: 端到端整合演示**，將學習：

1. **完整系統整合** - 結合前三個模組建立完整的治理系統
2. **生產級部署** - Docker 容器化、API 服務化
3. **性能優化** - 大規模文檔處理的性能調優
4. **監控與運維** - 系統健康檢查、日誌分析、告警機制

恭喜完成品質控制系統的學習！🎉 您現在擁有了建立企業級文檔品質管理系統的完整技能。